# Decomposing a word in morphemes

In [1]:
import numpy as np
import operator
from difflib import SequenceMatcher
!pip install python-Levenshtein
import Levenshtein
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import string
!pip install nltk
from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import train_test_split

In [2]:
!pip install anvil-uplink
import anvil.server
anvil.server.connect("2OGLA6PCLATEGZ36Y2E23JKT-CZK5OX76CFXB3UJ2")

  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment" as SERVER


## Derivational Morphemes

#Read data
Read data from RoMorphoDict (containing words, their stema and their description) and the prefixes and suffixes lists. 

In [3]:
with open('RoMorphoDict.txt', 'rt', encoding='ISO-8859-1') as data:  # read content from RoMorphoDict dataset
    train = data.read().split('\n')
with open('prefixe.txt', 'rt', encoding='ISO-8859-1') as pr:  # read the list of prefixes
    pref = pr.read().split('\n')
    pref = list(dict.fromkeys(pref))  # remove duplicates
with open('sufixe.txt', 'rt', encoding='ISO-8859-1') as su:  # read the list of suffixes
    suf = su.read().split('\n')
    suf = list(dict.fromkeys(suf))  # remove duplicates

In [4]:
print(train[0])
print(pref[0])
print(suf[0])

aalenián	aalenián	adj.m-n.sg.n-a.neart.
a
ie


# Preprocess data
Modify all special characters into characters that belong to the romanian alphabet

In [5]:
for i, elem in enumerate(train):
    train[i] = train[i].replace('á', 'a')
    train[i] = train[i].replace('é', 'e')
    train[i] = train[i].replace('ã', 'ă')
    train[i] = train[i].replace('í', 'i')
    train[i] = train[i].replace('ú', 'u')
    train[i] = train[i].replace('ä', 'ă')
    train[i] = train[i].replace('ó', 'o')
    train[i] = train[i].replace('þ', 'ț')
    train[i] = train[i].replace('º','ș')
    train[i] = train[i].replace('÷', 'â')

Split data from RoMorphoDidct dictionary into its components (lists of each word, lemma and description)

In [6]:
for i, elem in enumerate(train):
    train[i] = elem.split('\t')
    info = train[i][-1]
    train[i].append(info.split('.'))

In [7]:
train[0]

['aalenian',
 'aalenian',
 'adj.m-n.sg.n-a.neart.',
 ['adj', 'm-n', 'sg', 'n-a', 'neart', '']]

Extract all nouns from the dataset

In [8]:
train_data = []  # train data containing only the nouns
for i, elem in enumerate(train):
    if 's' in elem[3][0].split('/'):
        train_data.append(elem)

In [9]:
len(train_data)

376178

# Get all stems from RoMorphoDict dataset

In [10]:
stem_list = []  # list of all stems from RoMorphoDict dataset
for item in train:
    if len(item) > 2:
        stem_list.append(item[1])
stem_list = list(dict.fromkeys(stem_list))  # remove duplicates
print(stem_list)
print(len(stem_list))

['aalenian', 'aba', 'abac', 'abacă', 'abacterian', 'abager', 'abagerie', 'abagiu', 'abajur', 'abalienare', 'abandon', 'abandonare', 'abandonat', 'abandonic', 'abanos', 'abataj', 'abate', 'abatere', 'abatesă', 'abatiză', 'abator', 'abatorizare', 'abațial', 'abație', 'abaxial', 'abazic', 'abazie', 'abătut', 'abc', 'abcedat', 'abces', 'abderit', 'abdicare', 'abdomen', 'abdominal', 'abdominalgie', 'abdominoscop', 'abducere', 'abductor', 'abducție', 'abecedar', 'aberant', 'aberanță', 'aberativ', 'aberație', 'aberometru', 'aberoscop', 'abhorare', 'abientin', 'abietacee', 'abietin', 'abietinee', 'abil', 'abilitare', 'abilitat', 'abilitate', 'abiogen', 'abiogeneză', 'abiologie', 'abiotic', 'abiotrofie', 'abioză', 'abis', 'abisal', 'abisinian', 'abitație', 'abject', 'abjecție', 'abjudecare', 'abjurare', 'abjurator', 'abjurație', 'ablactare', 'ablactație', 'ablativ', 'ablațiune', 'ablaut', 'ablefarie', 'ablegat', 'ablepsie', 'ablutomanie', 'abluțiune', 'abnegare', 'abnegație', 'abnorm', 'abnormi

In [11]:
stem_list_nouns = []  # list of all stems of the nouns from RoMorphoDict dataset
for item in train_data:
    if len(item) > 2:
        stem_list_nouns.append(item[1])
stem_list_nouns = list(dict.fromkeys(stem_list_nouns))  # remove duplicates
print(stem_list_nouns)
print(len(stem_list_nouns))

['aba', 'abac', 'abacă', 'abager', 'abagerie', 'abagiu', 'abajur', 'abalienare', 'abandon', 'abandonare', 'abandonic', 'abanos', 'abataj', 'abate', 'abatere', 'abatesă', 'abatiză', 'abator', 'abatorizare', 'abație', 'abazie', 'abc', 'abces', 'abderit', 'abdicare', 'abdomen', 'abdominalgie', 'abdominoscop', 'abducere', 'abducție', 'abecedar', 'aberanță', 'aberație', 'aberometru', 'aberoscop', 'abhorare', 'abientin', 'abietacee', 'abietin', 'abietinee', 'abilitare', 'abilitate', 'abiogeneză', 'abiologie', 'abiotrofie', 'abioză', 'abis', 'abisinian', 'abitație', 'abjecție', 'abjudecare', 'abjurare', 'abjurație', 'ablactare', 'ablactație', 'ablativ', 'ablațiune', 'ablaut', 'ablefarie', 'ablegat', 'ablepsie', 'ablutomanie', 'abluțiune', 'abnegare', 'abnegație', 'abnormitate', 'abolire', 'aboliție', 'aboliționism', 'aboliționist', 'aboliționistă', 'abolițiune', 'abolla', 'abominațiune', 'abonament', 'abonare', 'abonat', 'abord', 'abordaj', 'abordare', 'abordor', 'aborigen', 'aborigenitate', 

Make dictionary from word to its stem

In [12]:
infl_morph = {}  # dictionary containing all words from RoMorphoDict and their stem
for item in train:
    if len(item) > 2:
        infl_morph[item[0]] = item[1]

# Get all derivational morphemes and the roots

In [13]:
def find_deriv_morph(word):
    '''
    Function that takes a word and derive it into prefix + root + suffix
    Input: the word to be analysed
    Output: the prefix, root and suffix of the word
    '''
    # if word in infl_morph:
    #     word_stem = infl_morph[word]  # get the stem of the word
    # else:
    #     similar_words = {}  # dictionary of all words that might be the stem of the input word
    #     word_stem = ""  # the most probable stem of the word
    #     for stem in stem_list:
    #         sim = SequenceMatcher(None, word, stem).ratio()  # compute the similarity between the stem and the input word
    #         if sim > 0.5:
    #             similar_words[stem] = sim
    #     if len(similar_words) > 0:
    #         word_stem = max(similar_words.items(), key=operator.itemgetter(1))[0]  # get the stem which is most probable to be the stem of the input word

    word_stem = word
    possible_prefs = []
    for p in pref:  # check if there are any possible prefixes in the input word
        if p == word_stem[:len(p)] and word_stem[len(p):] in stem_list:
            possible_prefs.append(p)

    possible_sufs = []  # check if there are any possible suffixes in the input word
    for s in suf:
        if s == word_stem[-(len(s)):]:
            possible_sufs.append(s)

    possible_prefs.sort(key=len)  # get the longest possible prefix
    if len(possible_prefs) > 0:
        most_sim_pref = possible_prefs[0]
    else:
        most_sim_pref = ""
    word_stem = word_stem[len(most_sim_pref):]  # remove the prefix from the input word

    possible_suf_combinations = {}  # find the most probable suffix from the list by comparing similarities between the word without the suffix with other words in the stem list
    for s in possible_sufs:
        for stem in stem_list:
            if word_stem[0] == stem[0]:
                sim = SequenceMatcher(None, word_stem[:-len(s)], stem).ratio()
                if sim > 0.8 :
                      possible_suf_combinations[(s, stem)] = sim
    most_sim_suf_ = []  # get all suffixes that led to the same similarity and choose the longest one
    if len(possible_suf_combinations) > 0:
        most_sim_suf = max(possible_suf_combinations.items(), key=operator.itemgetter(1))[0]
        for k in possible_suf_combinations:
            if possible_suf_combinations[k] == possible_suf_combinations[most_sim_suf]:
                most_sim_suf_.append(k)
        most_sim_suf_.sort(key=len)
        most_sim_suf = most_sim_suf_[0]
    else:
        most_sim_suf = ("", word_stem)
    return (most_sim_pref, most_sim_suf[1], most_sim_suf[0])

In [14]:
(prefix, root, suffix) = find_deriv_morph("iregularitate")
print("Prefix: ", prefix)
print("Root: ", root)
print("Suffix: ", suffix)

Prefix:  i
Root:  regular
Suffix:  itate


In [15]:
SequenceMatcher(None, "act", "at").ratio()

0.8

In [16]:
Levenshtein.distance("atent","atenție")

3

Get the root by having the word, stem, prefix and suffix

In [17]:
def find_root(word, word_stem, pref, suf):
    '''
    Function that takes a word, its stem, prefix and suffix and returns the root
    Input: the word, stem, prefix and suffix
    Output: the root
    '''
    word_stem = word_stem[len(pref):]  # remove the prefix from the input word

    possible_suf_combinations = {}  # find the most probable suffix from the list by comparing similarities between the word without the suffix with other words in the stem list
    possible_sufs = [suf]
    for s in possible_sufs:
        for stem in stem_list:
            if word_stem[0] == stem[0]:
                sim = SequenceMatcher(None, word_stem[:-len(s)], stem).ratio()
                if sim > 0.8 :
                      possible_suf_combinations[(s, stem)] = sim
    most_sim_suf_ = []  # get all suffixes that led to the same similarity and choose the longest one
    if len(possible_suf_combinations) > 0:
        most_sim_suf = max(possible_suf_combinations.items(), key=operator.itemgetter(1))[0]
        for k in possible_suf_combinations:
            if possible_suf_combinations[k] == possible_suf_combinations[most_sim_suf]:
                most_sim_suf_.append(k)
        most_sim_suf_.sort(key=len)
        most_sim_suf = most_sim_suf_[0]
    else:
        most_sim_suf = ("", word_stem)
    return most_sim_suf[1]

Get the list of all roots in  RoMorphoDict and construct a dictionary from each stem to its root, prefix and suffix

In [18]:
# roots = []  # list of all the roots in the RoMorphoDict
# deriv_morph = {}  # dictionary from each stem in RoMorphoDict to its root
# counter = 0
# for word in stem_list_nouns:
#     counter += 1
#     (prefix, root, suffix) = find_deriv_morph(word)
#     if root == word:
#         roots.append(word)
#     deriv_morph[word] = (prefix, root, suffix)

Constract a set of features formed by each word in RoMorphoDict and its stem and a set of labels (1 if the stem is diffrent from the root, meaning there are derivational morphemes, 0 otherwise)

In [19]:
# features = [["" for ind in range(2)] for j in range(len(train_data))]  # set of features formed by each word in RoMorphoDict and its stem
# labels = [0] * len(train_data)  # 1 if the stem is diffrent from the root, meaning there are derivational morphemes
# for indx, item in enumerate(train_data):
#     if len(item) > 2:
#         features[indx][0] = item[0]
#         features[indx][1] = item[1]
#         if deriv_morph[item[1]][1] != item[1]:  # the stem is diffrent from the root
#             labels[indx] = 1

Write the features to a file for faster access

In [20]:
# with open('output.txt', 'w') as writefile:  # write the features to a file
#   for indx, f in enumerate(features):
#     writefile.write("%s\t%s\n" % (f[0], f[1]))

Read fetures (stems and roots) from features file

In [21]:
with open('features.txt', 'rt', encoding='utf-8') as data:  # read features from file
    features = data.read().split('\n')
for i, elem in enumerate(features):
    features[i] = elem.split('\t')  # each feature will have a stem and its root

In [22]:
print(features[9])

['abreacție', 'reacție']


Generate a list of all the stems in the features file and generate the labels for the features (1 if stem is different from the root, 0 otherwise)

In [23]:
new_stem_list = []
labels = [0] * len(features)  # 1 if the stem is diffrent from the root, meaning there are derivational morphemes
for indx, item in enumerate(features):
    if item[0] != item[1]:  # the stem is diffrent from the root
        labels[indx] = 1
    new_stem_list.append(item[0])

Extract the prefix and suffix of a stem, if the root is provided

In [24]:
def find_prefixes_sufixes(word, root):
    '''
    Function that takes a word and derive it into prefix + root + suffix
    Input: the word to be analysed
    Output: the prefix, root and suffix of the word
    '''
    word_stem = word
    if word_stem == root:
        return ('', '')

    possible_prefs = []
    for p in pref:  # check if there are any possible prefixes in the input word
        remaining_word = word_stem[len(p):]
        if p == word_stem[:len(p)] and remaining_word[0:int(len(root)/2)] == root[0:int(len(root)/2)]:
            possible_prefs.append(p)

    possible_sufs = []  # check if there are any possible suffixes in the input word
    for s in suf:
        if s == word_stem[-(len(s)):]:
            possible_sufs.append(s)

    possible_prefs.sort(key=len)  # get the longest possible prefix
    if len(possible_prefs) > 0:
        most_sim_pref = possible_prefs[0]
    else:
        most_sim_pref = ""

    if word_stem[0:len(root)] == root:
        most_sim_pref = ""
        
    word_stem = word_stem[len(most_sim_pref):]  # remove the prefix from the input word

    possible_suf_combinations = {}  # find the most probable suffix from the list by comparing similarities between the word without the suffix with other words in the stem list
    for s in possible_sufs:
        sim = SequenceMatcher(None, word_stem[:-len(s)], root).ratio()
        if sim > 0.8 :
              possible_suf_combinations[(s, root)] = sim
    most_sim_suf_ = []  # get all suffixes that led to the same similarity and choose the longest one
    if len(possible_suf_combinations) > 0:
        most_sim_suf = max(possible_suf_combinations.items(), key=operator.itemgetter(1))[0]
        for k in possible_suf_combinations:
            if possible_suf_combinations[k] == possible_suf_combinations[most_sim_suf]:
                most_sim_suf_.append(k)
        most_sim_suf_.sort(key=len)
        most_sim_suf = most_sim_suf_[0]
    else:
        most_sim_suf = ("", root)
    return (most_sim_pref, most_sim_suf[0])

Construct a dictionary from each stem to its prefix, root and suffix

In [25]:
deriv_morph = {}  # dictionary from each stem in RoMorphoDict to its root, prefix and suffix
counter = 0
for indx, item in enumerate(features):
    counter += 1
    (prefix, suffix) = find_prefixes_sufixes(item[0], item[1])
    deriv_morph[item[0]] = (prefix, item[1], suffix)

Generate data as a list of words, roots, prefixes and suffixes and the labels (1 if the root is identical to the stem, 0 otherwise)

In [26]:
train_data[0]

['aba', 'aba', 's.f.sg.n-a.neart.', ['s', 'f', 'sg', 'n-a', 'neart', '']]

In [27]:
final_data = []  # will contain the features (words, their stem, prefix and suffix)
for item in train_data:
    if len(item) > 2 and item[1] in new_stem_list:
        final_data.append([item[0], item[1], deriv_morph[item[1]][0], deriv_morph[item[1]][2]])

In [28]:
labels = [0] * len(final_data)  # the label will be 1 if the root is identical with the stem
for i, item in enumerate(final_data):
    if len(item) > 2 and item[1] == deriv_morph[item[1]][1]:
        labels[i] = 1

In [29]:
labels[0]
final_data[0]

['aba', 'aba', '', '']

In [30]:
print(len(final_data))

37713


To make the features usable for a ML algoritm, we will generate the list of 3-grams for each word and will use it to construct a vocabulary of each possible letter combination and change the feature representation to make use of this dictionary.

In [33]:
new_features = [["" for ind in range(4)] for j in range(len(final_data))]  # this list will have the same words as in features matrix, but we will mark each begginging of a word with # and each end of a word with $
for indx, f in enumerate(final_data):
    new_features[indx][0] = '#' + f[0] + '$'
    new_features[indx][1] = '#' + f[1] + '$'
    new_features[indx][2] = f[2]
    new_features[indx][3] = f[3]

Construct a dictionary with all possible 1-grams, 2-grams and 3-grams from the dataset

In [34]:
def construct_vocab(features):  # construct the dictionary of all possible 1-grams, 2-grams and 3-grams
    ngram = {}
    index = 0
    for indx, f in enumerate(features):
        for i in range(2):
            for nr_ngrams in range(3):
                n = nr_ngrams + 1
                ngrams = [f[i][j:j+n] for j in range(len(f[i])-n+1)]
                for g in ngrams:
                    if g not in ngram:
                        ngram[g] = index
                        index += 1
    return ngram
vocabulary = construct_vocab(new_features)

In [35]:
print(len(vocabulary))

5580


Construct a vocabulary for all prefixes and suffixes

In [36]:
def pref_suf_vocab(features):
    vocab = {}  # vocabulary mapping all prefixes and suffixes with an unique integer
    index = 0
    for indx, f in enumerate(features):
        for i in range(2, 4):
              if f[i] not in vocab:
                  vocab[f[i]] = index
                  index += 1
    return vocab
pref_suf_vocabulary = pref_suf_vocab(new_features)

Construct a vocabulary for all prefixes

In [37]:
def pref_vocab(features):
    vocab = {}  # vocabulary mapping all prefixes with an unique integer
    index = 0
    for p in pref:
        if p not in vocab:
            vocab[p] = index
            index += 1
    return vocab
pref_vocabulary = pref_vocab(new_features)
pref_vocabulary[''] = -1

Construct a vocabulary for all suffixes

In [38]:
def suf_vocab(features):
    vocab = {}  # vocabulary mapping all suffixes with an unique integer
    index = 0
    for s in suf:
        if s not in vocab:
            vocab[s] = index
            index += 1
    return vocab
suf_vocabulary = suf_vocab(new_features)
suf_vocabulary[''] = -1

In [39]:
print(len(pref_suf_vocabulary))

182


In [40]:
print(pref_suf_vocabulary)

{'': 0, 'ic': 1, 'e': 2, 'ar': 3, 're': 4, 'ab': 5, 'or': 6, 'ent': 7, 'are': 8, 'a': 9, 'iv': 10, 'ie': 11, 'ism': 12, 'at': 13, 'ere': 14, 'ad': 15, 'tor': 16, 'ian': 17, 'ist': 18, 'aj': 19, 'itate': 20, 'it': 21, 'iu': 22, 'uri': 23, 'an': 24, 'ante': 25, 'anti': 26, 'al': 27, 'on': 28, 'gen': 29, 'atru': 30, 'antropo': 31, 'ier': 32, 'fon': 33, 'ut': 34, 'arhi': 35, 'ici': 36, 'oare': 37, 'tru': 38, 'auto': 39, 'iune': 40, 'il': 41, 'ire': 42, 'er': 43, 'bilitate': 44, 'avan': 45, 'ul': 46, 'in': 47, 'graf': 48, 'os': 49, 'ea': 50, 'ime': 51, 'bi': 52, 'bio': 53, 'og': 54, 'esc': 55, 'ee': 56, 'cefal': 57, 'el': 58, 'eag': 59, 'eu': 60, 'iciu': 61, 'ol': 62, 'gie': 63, 'et': 64, 'arh': 65, 'ant': 66, 'fil': 67, 'circum': 68, 'co': 69, 'ean': 70, 'itudine': 71, 'ibilitate': 72, 'con': 73, 'oriu': 74, 'contra': 75, 'andru': 76, 'cul': 77, 'crono': 78, 'cvadri': 79, 'de': 80, 'deca': 81, 'demo': 82, 'dent': 83, 'des': 84, 'dez': 85, 'di': 86, 'dia': 87, 'log': 88, 'dis': 89, 'ast': 9

Features list containing the vocabulary for the stem, the root, the prefix and suffix encodings.

In [41]:
all_features = [[0 for i in range(len(vocabulary) * 2 + len(pref_suf_vocabulary) * 2)] for j in range(len(final_data))]
for indx, f in enumerate(new_features):
        for i in range(2):
            for nr_ngrams in range(3):
                n = nr_ngrams + 1
                ngrams = [f[i][j:j+n] for j in range(len(f[i])-n+1)]
                for g in ngrams:
                    if g in vocabulary:
                        all_features[indx][vocabulary[g] + len(vocabulary) * i] = 1
        for i in range(2, 4):
            all_features[indx][pref_suf_vocabulary[f[i]] + len(vocabulary) * 2 + len(pref_suf_vocabulary) * (i-2)] = 1
all_features_train, all_features_test, all_labels_train, all_labels_test = train_test_split(all_features, labels, test_size=0.25, random_state=42)
final_data_train, final_data_test, final_labels_train, final_labels_test = train_test_split(final_data, labels, test_size=0.25, random_state=42)

Features list containing the vocabulary for the stem and the root encodings. The labels consists of the suffix encoding of the word.

In [42]:
all_features_suf = [[0 for i in range(len(vocabulary) * 1)] for j in range(len(final_data))]
all_labels_suf = [0] * len(final_data)
for indx, f in enumerate(new_features):
        for i in range(1):
            for nr_ngrams in range(3):
                n = nr_ngrams + 1
                ngrams = [f[i+1][j:j+n] for j in range(len(f[i+1])-n+1)]
                for g in ngrams:
                    if g in vocabulary:
                        all_features_suf[indx][vocabulary[g] + len(vocabulary) * i] = 1
        all_labels_suf[indx] = suf_vocabulary[f[3]]
all_features_suf_train, all_features_suf_test, all_labels_suf_train, all_labels_suf_test = train_test_split(all_features_suf, all_labels_suf, test_size=0.25, random_state=42)

In [43]:
print(all_labels_suf)

[-1, -1, -1, -1, -1, -1, -1, -1, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 13, 13, 13, 13, 13, 13, 13, 13, 33, 33, 33, 33, 33, 33, 33, 33, 157, 157, 157, 157, 157, 157, 157, 157, 157, 157, 157, 157, 157, 157, 157, 157, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 7, 7, 7, 7, 7, 7, 7, 7, 13, 13, 13, 13, 13, 13, 13, 13, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 29, 29, 29, 29, 29, 29, 29, 29, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, -1, 157, 157, 157, 157, 13, 13, 13, 13, 157, 157, 157, 157, 157, 157, 157, 157, 157, 157, 157, 157, 157, 157, 157, 157, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 157, 157, 157, 157, 157, 157, 157, 157, -1, -1, -1, -1, -1, -1, -1, -1, 7, 7, 7, 7, 7, 

Features list containing the vocabulary for the stem and the root encodings. The labels consists of the prefix encoding of the word.

In [44]:
all_features_pref = [[0 for i in range(len(vocabulary) * 1)] for j in range(len(final_data))]
all_labels_pref = [0] * len(final_data)
for indx, f in enumerate(new_features):
        for i in range(1):
            for nr_ngrams in range(3):
                n = nr_ngrams + 1
                ngrams = [f[i+1][j:j+n] for j in range(len(f[i+1])-n+1)]
                for g in ngrams:
                    if g in vocabulary:
                        all_features_pref[indx][vocabulary[g] + len(vocabulary) * i] = 1
        all_labels_pref[indx] = pref_vocabulary[f[2]]
all_features_pref_train, all_features_pref_test, all_labels_pref_train, all_labels_pref_test = train_test_split(all_features_pref, all_labels_pref, test_size=0.25, random_state=42)

In [45]:
print(all_labels_pref)

[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 2, 2, 2, 2, 2, 2, 2, 2, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 2, 2, 2, 2, 2, 2, 2, 2, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, 0, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, 0, 0, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,

Implement and test the model in a 5-fold cross val setting

In [46]:
# clf = SVC()
# scores_ = cross_val_score(clf, all_features, labels, cv=5)
# print(scores_.mean(), scores_.std())

In [47]:
# clf = SVC().fit(all_features, labels)
# pred_labels_ = clf.predict(all_features)
# print(accuracy_score(pred_labels_, labels))

In [48]:
# pred_labels_[:100]

Clasify the prefixes of a word using SVC classifier

In [50]:
clf = SVC()
scores_pref = cross_val_score(clf, all_features_pref[:10000], all_labels_pref[:10000], cv=5)
print(scores_pref.mean(), scores_pref.std())

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  UserWarning,


0.8615 0.08028698524667621


In [51]:
model_pref = SVC().fit(all_features_pref, all_labels_pref)

In [ ]:
# model_pref = SVC().fit(all_features_pref_train, all_labels_pref_train)
# pred_labels_pref = model_pref.predict(all_features_pref_test)
# print(accuracy_score(pred_labels_pref, all_labels_pref_test))

Clasify the suffixes of a word using SVC classifier

In [52]:
clf = SVC()
scores_suf = cross_val_score(clf, all_features_suf[:10000], all_labels_suf[:10000], cv=5)
print(scores_suf.mean(), scores_suf.std())

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,


0.8017 0.04247658178337799


In [53]:
model_suf = SVC().fit(all_features_suf, all_labels_suf)

In [54]:
# model_suf = SVC().fit(all_features_suf_train, all_labels_suf_train)
# pred_labels_suf = model_suf.predict(all_features_suf_test)
# print(accuracy_score(pred_labels_suf, all_labels_suf_test))

In [55]:
def convert_into_features(word):
      feat = [0 for i in range(len(vocabulary))]
      for nr_ngrams in range(3):
          n = nr_ngrams + 1
          ngrams = [word[j:j+n] for j in range(len(word)-n+1)]
          for g in ngrams:
              if g in vocabulary:
                  feat[vocabulary[g]] = 1
      return feat

In [56]:
def morpheme_deriv(word_stem):
    word = convert_into_features(word_stem)
    [pred_pref] = model_pref.predict([word])
    [pred_suf] = model_suf.predict([word])
    for p in pref_vocabulary:
        if pref_vocabulary[p] == pred_pref:
            prefix = p
    for s in suf_vocabulary:
        if suf_vocabulary[s] == pred_suf:
            suffix = s
    root = find_root(word_stem, word_stem, prefix, suffix)
    return (prefix, root, suffix)

In [57]:
(prefix, root, suffix) = morpheme_deriv("nepopularitate")
print("The prefix is: ", prefix)
print("The root is: ", root)
print("The suffix is: ", suffix)

The prefix is:  ne
The root is:  popular
The suffix is:  itate


## Inflectional morphemes

In [58]:
f = open('RoMorphoDict.txt', "r", encoding='latin-1')
lines_train=f.readlines()

In [59]:
print(lines_train[:10])

['aalenián\taalenián\tadj.m-n.sg.n-a.neart.\n', 'aalenián\taalenián\tadj.m-n.sg.g-d.neart.\n', 'aaleniéni\taalenián\tadj.m.pl.n-a.neart.\n', 'aaleniéni\taalenián\tadj.m.pl.g-d.neart.\n', 'aaleniánã\taalenián\tadj.f.sg.n-a.neart.\n', 'aaleniéne\taalenián\tadj.f.sg.g-d.neart.\n', 'aaleniéne\taalenián\tadj.f-n.pl.n-a.neart.\n', 'aaleniéne\taalenián\tadj.f-n.pl.g-d.neart.\n', 'abá\tabá\ts.f.sg.n-a.neart.\n', 'abále\tabá\ts.f.sg.g-d.neart.\n']


In [60]:
for i in range(len(lines_train)):
  lines_train[i] = lines_train[i].replace('á', 'a')
  lines_train[i] = lines_train[i].replace('é', 'e')
  lines_train[i] = lines_train[i].replace('ã', 'a')
  lines_train[i] = lines_train[i].replace('í', 'i')
  lines_train[i] = lines_train[i].replace('ó', 'o')
  lines_train[i] = lines_train[i].replace('ä', 'a')
  lines_train[i] = lines_train[i].replace('ú', 'u')
  lines_train[i] = lines_train[i].replace('â', 'a')
  lines_train[i] = lines_train[i].replace('î', 'i')
  lines_train[i] = lines_train[i].replace('ï', 'i')
  lines_train[i] = lines_train[i].replace('ö', 'o')
  lines_train[i] = lines_train[i].replace('÷', 'a')
  lines_train[i] = lines_train[i].replace('ü', 'u')
  lines_train[i] = lines_train[i].replace('þ', 't')
  lines_train[i] = lines_train[i].replace('º', 's')
  lines_train[i] = lines_train[i].replace('_', '-')                                  #jalele_de_camp jale_de_camp

In [61]:
print(lines_train[100:500])

['abandonic\tabandonic\ts/adj.m/m-n.sg.n-a.neart.\n', 'abandonic\tabandonic\ts/adj.m/m-n.sg.g-d.neart.\n', 'abandonicul\tabandonic\ts.m.sg.n-a.art.\n', 'abandonicului\tabandonic\ts.m.sg.g-d.art.\n', 'abandonici\tabandonic\ts/adj.m.pl.n-a.neart.\n', 'abandonici\tabandonic\ts/adj.m.pl.g-d.neart.\n', 'abandonicii\tabandonic\ts.m.pl.n-a.art.\n', 'abandonicilor\tabandonic\ts.m.pl.g-d.art.\n', 'abandonica\tabandonic\ts/adj.f.sg.n-a.neart.\n', 'abandonice\tabandonic\ts/adj.f.sg.g-d.neart.\n', 'abandonica\tabandonic\ts.f.sg.n-a.art.\n', 'abandonicei\tabandonic\ts.f.sg.g-d.art.\n', 'abandonice\tabandonic\ts/adj.f/f-n.pl.n-a.neart.\n', 'abandonice\tabandonic\ts/adj.f/f-n.pl.g-d.neart.\n', 'abandonicele\tabandonic\ts.f.pl.n-a.art.\n', 'abandonicelor\tabandonic\ts.f.pl.g-d.art.\n', 'abanos\tabanos\ts.m.sg.n-a.neart.\n', 'abanos\tabanos\ts.m.sg.g-d.neart.\n', 'abanosul\tabanos\ts.m.sg.n-a.art.\n', 'abanosului\tabanos\ts.m.sg.g-d.art.\n', 'abanosi\tabanos\ts.m.pl.n-a.neart.\n', 'abanosi\tabanos\ts.m

In [62]:
train_separated = []
for line in lines_train:
  line_aux = line[:-1]           #eliminate "\n" from the end of the line
  train_separated.append(line_aux.split('\t'))

In [63]:
print(train_separated[9:20])

[['abale', 'aba', 's.f.sg.g-d.neart.'], ['abaua', 'aba', 's.f.sg.n-a.art.'], ['abalei', 'aba', 's.f.sg.g-d.art.'], ['abale', 'aba', 's.f.pl.n-a.neart.'], ['abale', 'aba', 's.f.pl.g-d.neart.'], ['abalele', 'aba', 's.f.pl.n-a.art.'], ['abalelor', 'aba', 's.f.pl.g-d.art.'], ['abac', 'abac', 's.n.sg.n-a.neart.'], ['abac', 'abac', 's.n.sg.g-d.neart.'], ['abacul', 'abac', 's.n.sg.n-a.art.'], ['abacului', 'abac', 's.n.sg.g-d.art.']]


In [64]:
stemmer = SnowballStemmer("romanian")

In [65]:
print(stemmer.stem("aalenieni"))

aalenien


In [66]:
first_column_aux = []
second_column_aux = []
third_column_aux = []
for line in train_separated:
  first_column_aux.append(line[0])
  second_column_aux.append(line[1])
  third_column_aux.append(line[2])

In [67]:
print("third_column_aux: {}".format(third_column_aux[:10]))

third_column_aux: ['adj.m-n.sg.n-a.neart.', 'adj.m-n.sg.g-d.neart.', 'adj.m.pl.n-a.neart.', 'adj.m.pl.g-d.neart.', 'adj.f.sg.n-a.neart.', 'adj.f.sg.g-d.neart.', 'adj.f-n.pl.n-a.neart.', 'adj.f-n.pl.g-d.neart.', 's.f.sg.n-a.neart.', 's.f.sg.g-d.neart.']


In [68]:
tag = []            #tag = part of speech
list_ = []
for i in range(len(first_column_aux)):
  list_.append(third_column_aux[i].split('.'))
  tag.append(list_[-1][0].split('/'))

In [69]:
print("tag: {}".format(tag[:10]))
print("list_: {}".format(list_[:10]))

tag: [['adj'], ['adj'], ['adj'], ['adj'], ['adj'], ['adj'], ['adj'], ['adj'], ['s'], ['s']]
list_: [['adj', 'm-n', 'sg', 'n-a', 'neart', ''], ['adj', 'm-n', 'sg', 'g-d', 'neart', ''], ['adj', 'm', 'pl', 'n-a', 'neart', ''], ['adj', 'm', 'pl', 'g-d', 'neart', ''], ['adj', 'f', 'sg', 'n-a', 'neart', ''], ['adj', 'f', 'sg', 'g-d', 'neart', ''], ['adj', 'f-n', 'pl', 'n-a', 'neart', ''], ['adj', 'f-n', 'pl', 'g-d', 'neart', ''], ['s', 'f', 'sg', 'n-a', 'neart', ''], ['s', 'f', 'sg', 'g-d', 'neart', '']]


In [70]:
#first_column: [['aalenian', 'aalenian', 'aalenieni', 'aalenieni', ...], ['aba', 'abale', 'abaua', 'abalei', ...], ...]
#second_column: [['aalenian', 'aalenian', 'aalenian', 'aalenian', 'aalenian', 'aalenian', 'aalenian', 'aalenian'], ...]
#stem: [['aalenian', 'aalenian', 'aalenien', 'aalenien', ...], ['aba', 'abal', 'abau', 'abale', ...], ...]
#third_column: [['s.f.sg.n-a.neart.', 's.f.sg.g-d.neart.', ...], ['s.n.sg.n-a.neart.', 's.n.sg.g-d.neart.', ...], ...]
#count: [['sg', 'sg', 'sg', 'sg', 'pl', 'pl', 'pl', 'pl'], ['sg', 'sg', 'sg', 'sg', 'pl', 'pl', 'pl', 'pl'], ...]
#article: [['neart', 'neart', 'art', 'art', ...], ['neart', 'neart', 'art', 'art', ...], ...]
#gen: [[['f'], ['f'], ['f'], ['f'], ['f'], ['f'], ['f'], ['f']], [['n'], ['n'], ['n'], ['n'], ['n'], ['n'], ['n'], ['n']], ...]
#caz: [['n-a', 'g-d', 'n-a', 'g-d', 'n-a', 'g-d', 'n-a', 'g-d'], ['n-a', 'g-d', 'n-a', 'g-d', 'n-a', 'g-d', 'n-a', 'g-d'], ...]

first_column = []
second_column = []
third_column = []
stem = []
count = []                                                #singular or plural
article = []
gen = []
caz = []                                                 #gentiv-dativ/nominativ-dativ

i = 0
while i < len(first_column_aux):
  word = second_column_aux[i]
  new_first_column = []
  new_second_column = []
  new_third_column = []
  new_stem = []
  new_article = []
  new_count = []
  new_gen = []
  new_caz = []
  while i < len(second_column_aux) and word == second_column_aux[i]:
    if 's' in tag[i]:
      new_first_column.append(first_column_aux[i])
      new_second_column.append(second_column_aux[i])
      new_third_column.append(third_column_aux[i])
      new_stem.append(stemmer.stem(first_column_aux[i]))
      new_count.append(list_[i][2])
      new_gen.append(list_[i][1])
      new_article.append(list_[i][4])
      new_caz.append(list_[i][3])
      #print(third_column_aux[i].split('.')[0].split('/'))
    i += 1
  if new_stem != []:                                                 #if all lists from above are not []
    first_column.append(new_first_column.copy())
    second_column.append(new_second_column.copy())
    third_column.append(new_third_column.copy())
    stem.append(new_stem.copy())
    article.append(new_article.copy())
    count.append(new_count.copy())
    gen.append(new_gen)
    caz.append(new_caz)

In [71]:
print("first_column: {}".format(first_column[:10]))
print("second_column: {}".format(second_column[:10]))
print("third_column: {}".format(third_column[:10]))
print("stem: {}".format(stem[:10]))
print("count: {}".format(count[:10]))
print("article: {}".format(article[:10]))
print("gen: {}".format(gen[:10]))
print("caz: {}".format(caz[:10]))

first_column: [['aba', 'abale', 'abaua', 'abalei', 'abale', 'abale', 'abalele', 'abalelor'], ['abac', 'abac', 'abacul', 'abacului', 'abace', 'abace', 'abacele', 'abacelor'], ['abaca', 'abace', 'abaca', 'abacei', 'abace', 'abace', 'abacele', 'abacelor'], ['abager', 'abager', 'abagerul', 'abagerului', 'abageri', 'abageri', 'abagerii', 'abagerilor'], ['abagerie', 'abagerii', 'abageria', 'abageriei', 'abagerii', 'abagerii', 'abageriile', 'abageriilor'], ['abagiu', 'abagiu', 'abagiul', 'abagiului', 'abagii', 'abagii', 'abagiii', 'abagiilor'], ['abajur', 'abajur', 'abajurul', 'abajurului', 'abajururi', 'abajururi', 'abajururile', 'abajururilor'], ['abalienare', 'abalienari', 'abalienarea', 'abalienarii'], ['abandon', 'abandon', 'abandonul', 'abandonului', 'abandonuri', 'abandonuri', 'abandonurile', 'abandonurilor'], ['abandonare', 'abandonari', 'abandonarea', 'abandonarii', 'abandonari', 'abandonari', 'abandonarile', 'abandonarilor']]
second_column: [['aba', 'aba', 'aba', 'aba', 'aba', 'aba'

In [72]:
def gd(first_col, second_col):
  compus = first_col.split("-")
  flectiv = ''
  if len(compus) > 1:
    first_col = compus[0] 
  if first_col == second_col:
    return first_col, ''                                
  elif len(first_col) > 5 and first_col[-6:] == 'ilelor':
    flectiv = 'ilelor'  
  elif len(first_col) > 5 and first_col[-6:] == 'urilor':
    flectiv = 'urilor' 
  elif first_col[-5:] == 'lelor':
    flectiv = 'lelor'
  elif first_col[-5:] == 'urile':
    flectiv = 'urile'
  elif first_col[-5:] == 'lilor':
    flectiv = 'lilor'
  elif first_col[-4:] == 'elor':
    flectiv = 'elor'
  elif first_col[-4:] == 'ulor':
    flectiv = 'ulor'
  elif first_col[-4:] == 'alor':
    flectiv = 'alor'
  elif first_col[-4:] == 'ilor':
    flectiv = 'ilor'
  elif first_col[-4:] == 'ului': #and second_col[-4] != 'u':
    flectiv = 'ului'
  elif first_col[-3:] == 'lei': #and second_col[-3] != 'l':
    flectiv = 'lei'
  elif first_col[-3:] == 'lui': #and second_col[-3:-1] != 'lu':
    flectiv = 'lui'
  elif first_col[-3:] == 'uri':
    flectiv = 'uri'
  elif first_col[-3:] == 'iii':
    flectiv = 'iii'
  elif first_col[-3:] == 'lor':
    flectiv = 'lor'
  elif first_col[-2:] == 'ei':
    flectiv = 'ei'
  elif first_col[-2:] == 'le':
    flectiv = 'le'
  elif first_col[-2:] == 'ii':
    flectiv = 'ii'
  elif first_col[-2:] == 'li':
    flectiv = 'li'
  elif first_col[-1] == 'e':
    flectiv = 'e'
  elif first_col[-1] == 'a':
    flectiv = 'a'
  elif first_col[-1] == 'i':
    flectiv = 'i'
  else:
    return -1, -1

  expr_root = first_col[:-len(flectiv)]
  if len(compus) > 1 and len(compus[-1]) > 1:
    compus2 = second_col.split("-")
    first_col2, flectiv2 = na(compus[-1], compus2[-1])
    if first_col2 == -1:
      return -1, -1
    else:
      expr_root = [first_col[:-len(flectiv)]] + compus[1:-1] + [first_col2] 
      flectiv = [flectiv, flectiv2]

  return expr_root, flectiv

In [73]:
def na(first_col, second_col):
  compus = first_col.split("-")
  flectiv = ''
  if len(compus) > 1:
    first_col = compus[0]  
  if first_col == second_col:
    return first_col, ''
  elif len(first_col) > 5 and first_col[-6:] == 'urilor':
    flectiv = 'urilor'
  elif first_col[-5:] == 'ilele':
    flectiv = 'ilele'
  elif first_col[-5:] == 'urile':
    flectiv = 'urile'
  elif first_col[-4:] == 'lele':
    flectiv = 'lele'
  elif first_col[-4:] == 'ilor':
    flectiv = 'ilor'
  elif first_col[-3:] == 'ele':
    flectiv = 'ele'
  elif first_col[-3:] == 'ile':
    flectiv = 'ile'      
  elif first_col[-3:] == 'ule':
    flectiv = 'ule'   
  elif first_col[-3:] == 'ale':
    flectiv = 'ale' 
  elif first_col[-3:] == 'uri':
    flectiv = 'uri'   
  elif first_col[-3:] == 'iii':
    flectiv = 'iii' 
 # elif first_col[-3:] == 'aia':
  #  flectiv = 'aia'                          
  elif first_col[-2:] == 'ua':
    flectiv = 'ua'
  elif first_col[-2:] == 'ul':
    flectiv = 'ul'
  elif first_col[-2:] == 'le':
    flectiv = 'le'
  elif first_col[-2:] == 'li':
    flectiv = 'li'
  #elif first_col[-2:] == 'ea':
   # flectiv = 'ea'
  elif first_col[-2:] == 'ei':
    flectiv ='ei'
  elif first_col[-2:] == 'ii':
    flectiv ='ii'
  elif first_col[-1] == 'o':
    flectiv = 'o'
  elif first_col[-1] == 'a':
    flectiv = 'a'
  elif first_col[-1] == 'e':
    flectiv = 'e'
  elif first_col[-1] == 'i':
    flectiv = 'i'
  elif first_col[-1] == 'l':
    flectiv = 'l'
  else:
    return -1, -1

  expr_root = first_col[:-len(flectiv)]
  if len(compus) > 1 and len(compus[-1]) > 1:
    compus2 = second_col.split("-")
    first_col2, flectiv2 = na(compus[-1], compus2[-1])
    if first_col2 == -1:
      return -1, -1
    else:
      expr_root = [first_col[:-len(flectiv)]] + compus[1:-1] + [first_col2] 
      flectiv = [flectiv, flectiv2]
  return expr_root, flectiv

In [74]:
def vc(first_col, second_col):
  flectiv = ''
  if first_col[-3:] == 'ule':
    flectiv = 'ule'
  elif first_col[-4:] == 'elor':
    flectiv = 'elor'
  elif first_col[-4:] == 'ilor':
    flectiv = 'ilor'
  elif first_col[-1] == 'e':
    flectiv = 'e'
  elif first_col[-1] == 'o':
    flectiv = 'o'
  else:
    return -1, -1

  expr_root = first_col[:-len(flectiv)]
  return expr_root, flectiv

In [75]:
import copy

For examples like first_column - "zbarnaitoare", second_column (correct stem) - "zbarnaitor",
stem (with the stemmer) - "zbarnait" (incorrect), we want the inflectional morphem = "-e". "o" from second column becomes "oa" in the first column.

In [76]:
expr_morfem = []                                            #flectiv
expr_root = []
first_column_copy = copy.deepcopy(first_column)
second_column_copy = copy.deepcopy(second_column)
for i in range(len(first_column)):
  expr_morfem_new = []
  expr_root_new = []
  flectiv = -1
  for j in range(len(first_column[i])):
    if first_column[i][j] == second_column[i][j]:
      flectiv = ''
      root = first_column[i][j]
    elif len(stem[i][j]) == len(second_column[i][j]):
      flectiv = first_column[i][j][len(stem[i][j]):]
      root = first_column[i][j][:len(stem[i][j])]
    elif len(stem[i][j]) != len(second_column[i][j]):  
      if caz[i][j] == 'g-d':
        root, flectiv = gd(first_column[i][j], second_column[i][j])
      elif caz[i][j] == 'n-a':
        root, flectiv = na(first_column[i][j], second_column[i][j])
      elif caz[i][j] == 'vc':
        root, flectiv = vc(first_column[i][j], second_column[i][j])
    if flectiv != -1:
      expr_root_new.append(root)
      expr_morfem_new.append(flectiv)
    else:
      first_column_copy[i].remove(first_column[i][j])
      second_column_copy[i].remove(second_column[i][j])
  expr_morfem.append(expr_morfem_new.copy())
  expr_root.append(expr_root_new.copy())

In [77]:
print(first_column_copy[:10])
print(second_column_copy[:10])
print(expr_root[:10])
print(expr_morfem[:10])

[['aba', 'abale', 'abaua', 'abalei', 'abale', 'abale', 'abalele', 'abalelor'], ['abac', 'abac', 'abacul', 'abacului', 'abace', 'abace', 'abacele', 'abacelor'], ['abaca', 'abace', 'abaca', 'abacei', 'abace', 'abace', 'abacele', 'abacelor'], ['abager', 'abager', 'abagerul', 'abagerului', 'abageri', 'abageri', 'abagerii', 'abagerilor'], ['abagerie', 'abagerii', 'abageria', 'abageriei', 'abagerii', 'abagerii', 'abageriile', 'abageriilor'], ['abagiu', 'abagiu', 'abagiul', 'abagiului', 'abagii', 'abagii', 'abagiii', 'abagiilor'], ['abajur', 'abajur', 'abajurul', 'abajurului', 'abajururi', 'abajururi', 'abajururile', 'abajururilor'], ['abalienare', 'abalienari', 'abalienarea', 'abalienarii'], ['abandon', 'abandon', 'abandonul', 'abandonului', 'abandonuri', 'abandonuri', 'abandonurile', 'abandonurilor'], ['abandonare', 'abandonari', 'abandonarea', 'abandonarii', 'abandonari', 'abandonari', 'abandonarile', 'abandonarilor']]
[['aba', 'aba', 'aba', 'aba', 'aba', 'aba', 'aba', 'aba'], ['abac', 'ab

In [78]:
print(expr_morfem[:10])

[['', 'le', 'ua', 'lei', 'ale', 'le', 'lele', 'lelor'], ['', '', 'ul', 'ului', 'e', 'e', 'ele', 'elor'], ['', 'e', '', 'i', 'e', 'e', 'ele', 'elor'], ['', '', 'ul', 'ului', 'i', 'i', 'ii', 'ilor'], ['', 'ii', 'a', 'ei', 'ii', 'ii', 'ile', 'ilor'], ['', '', 'ul', 'ului', 'ii', 'ii', 'i', 'ilor'], ['', '', 'ul', 'ului', 'uri', 'uri', 'urile', 'urilor'], ['', 'i', 'a', 'ii'], ['', '', 'ul', 'ului', 'uri', 'uri', 'urile', 'urilor'], ['', 'i', 'a', 'ii', 'i', 'i', 'ile', 'ilor']]


In [79]:
couples = []
for i in range(len(first_column_copy)):
  for j in range(len(first_column_copy[i])):
    couples.append((i, j))

In [80]:
data_train, data_test, labels_train, labels_test = train_test_split(couples, couples, test_size=0.0001, random_state=42)

In [81]:
for i, j in couples:
  if (i,j) in labels_test:
      if not isinstance(expr_morfem[i][j], list):
        print(first_column_copy[i][j], second_column_copy[i][j], expr_root[i][j] + "-" + expr_morfem[i][j])
      else:
        print(first_column_copy[i][j], second_column_copy[i][j], 
              expr_root[i][j][0] + expr_morfem[i][j][0] + "-" + "-".join(expr_root[i][j][1:-1]) + "-" + expr_root[i][j][-1] + "-" + expr_morfem[i][j][-1])

alchimistului alchimist alchimist-ului
argentinienii argentinian argentinien-ii
asparaginei asparagina asparagine-i
audioprotector audioprotector audioprotector-
beznele bezna bezn-ele
carciumarite carciumarita carciumarit-e
cimbale cimbal cimbal-e
cineproiectoare cineproiector cineproiectoar-e
ciomagar ciomagar ciomagar-
cordeliere cordeliera cordelier-e
crucile cruce cruc-ile
dainuirea dainuire dainuire-a
degudronari degudronare degudronar-i
democratismului democratism democratism-ului
destoiniciei destoinicie destoinici-ei
detalieri detaliere detalier-i
dialog dialog dialog-
editari editare editar-i
epitome epitoma epitom-e
eponjul eponj eponj-ul
faretre faretra faretr-e
finantarea finantare finantare-a
grapari grapare grapar-i
harababurile harababura harabab-urile
jaragai jaragai jaragai-
mecanicisti mecanicist mecanicist-i
neofitei neofita neofite-i
ojog ojog ojog-
orande oranda orand-e
pescarii pescarie pescar-ii
probelele probela probe-lele
rapuitoare rapuitoare rapuitoare-
reti

In [82]:
def morphem(morfem):
  morfeme = ''
  if morfem == 'ilelor':
    morfeme = ['ile', 'lor']
  elif morfem == 'urilor':
    morfeme = ['uri', 'lor']
  elif morfem == 'lelor':
    morfeme = ['le', 'lor']
  elif morfem == 'urile':
    morfeme = ['uri', 'le']
  elif morfem == 'lilor':
    morfeme = ['li', 'lor']
  elif morfem == 'elor':
    morfeme = ['e', 'lor']
  elif morfem == 'ulor':
    morfeme = ['u', 'lor']
  elif morfem == 'alor':
    morfeme = ['a', 'lor']
  elif morfem == 'ilor':
    morfeme = ['i', 'lor']
  elif morfem == 'ului':
    morfeme = ['u', 'lui']
  elif morfem == 'lei':
    morfeme = ['lei']
  elif morfem == 'lui':
    morfeme = ['lui']
  elif morfem == 'uri':
    morfeme = ['uri']
  elif morfem == 'iii':
    morfeme = ['i', 'i']
  elif morfem == 'lor':
    morfeme = ['lor']
  elif morfem == 'ei':
    morfeme = ['ei']
  elif morfem == 'le':
    morfeme = ['le']
  elif morfem == 'ii':
    morfeme = ['i', 'i']
  elif morfem == 'li':
    morfeme = ['li']
  elif morfem == 'e':
    morfeme = ['e']
  elif morfem == 'a':
    morfeme = ['a']
  elif morfem == 'i':
    morfeme = ['i']
  elif morfem == 'ilele':
    morfeme = ['ile', 'le']
  elif morfem == 'urile':
    morfeme = ['uri', 'le']
  elif morfem == 'lele':
    morfeme = ['le', 'le']
  elif morfem == 'ele':
    morfeme = ['e', 'le']
  elif morfem == 'ile':
    morfeme = ['i', 'le']
  elif morfem == 'ule':
    morfeme = ['u', 'le']
  elif morfem == 'ale':
    morfeme = ['a', 'le']
  elif morfem == 'ua':
    morfeme = ['ua']
  elif morfem == 'ul':
    morfeme = ['ul']
  elif morfem == 'o':
    morfeme = ['o']
  elif morfem == 'l':
    morfeme = ['l']
  elif morfem == 'ule':
    morfeme = ['ule']
  elif isinstance(morfem, list):
    morfeme = [morphem(x) for x in morfem]
  else:
    morfeme = ['']
  return morfeme

In [83]:
morfeme_flex = []
for i in range(len(expr_morfem)):
  morfeme_flex_new = []
  for flectiv in expr_morfem[i]:
    morfeme = morphem(flectiv)
    morfeme_flex_new.append(morfeme)
  morfeme_flex.append(morfeme_flex_new)

In [84]:
print(morfeme_flex[:10])

[[[''], ['le'], ['ua'], ['lei'], ['a', 'le'], ['le'], ['le', 'le'], ['le', 'lor']], [[''], [''], ['ul'], ['u', 'lui'], ['e'], ['e'], ['e', 'le'], ['e', 'lor']], [[''], ['e'], [''], ['i'], ['e'], ['e'], ['e', 'le'], ['e', 'lor']], [[''], [''], ['ul'], ['u', 'lui'], ['i'], ['i'], ['i', 'i'], ['i', 'lor']], [[''], ['i', 'i'], ['a'], ['ei'], ['i', 'i'], ['i', 'i'], ['i', 'le'], ['i', 'lor']], [[''], [''], ['ul'], ['u', 'lui'], ['i', 'i'], ['i', 'i'], ['i'], ['i', 'lor']], [[''], [''], ['ul'], ['u', 'lui'], ['uri'], ['uri'], ['uri', 'le'], ['uri', 'lor']], [[''], ['i'], ['a'], ['i', 'i']], [[''], [''], ['ul'], ['u', 'lui'], ['uri'], ['uri'], ['uri', 'le'], ['uri', 'lor']], [[''], ['i'], ['a'], ['i', 'i'], ['i'], ['i'], ['i', 'le'], ['i', 'lor']]]


In [85]:
def infl_morphem(word, first_col=first_column_copy, morphemes=morfeme_flex, stems=expr_root):
  word = word.replace('ș', 's')
  word = word.replace('ț', 't')
  word = word.replace('ă', 'a')
  word = word.replace('î', 'i')
  word = word.replace('â', 'a')

  stem = word
  infl_morphemes = ['']

  for i in range(len(first_col)):
    for j in range(len(first_col[i])):
      if word == first_col[i][j]:
        stem = stems[i][j]
        infl_morphemes = morphemes[i][j]

  return stem, infl_morphemes

In [86]:
print(infl_morphem('lazaretul'))

('lazaret', ['ul'])


In [87]:
@anvil.server.callable
def morphemes(word):
    stem, infl_morphemes = infl_morphem(word)
    (prefix, root, suffix) = morpheme_deriv(stem)
    return prefix, root, suffix, infl_morphemes

In [88]:
anvil.server.wait_forever()